In [17]:
import os
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
from torch.cuda.amp import GradScaler
#!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
context = 15
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # TODO: Please try different architectures
        in_size = (2*context+1)*13
        layers = [
            nn.Linear(in_size,512),
            nn.ReLU(),
            nn.Linear(512,40) # Earlier it was 40
        ]
        self.laysers = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.laysers(A0)
        return x

In [18]:
## Context 2##
context = 32
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # TODO: Please try different architectures
        in_size = (2*context+1)*13
        layers = [
            nn.Linear(in_size,512),
            nn.BatchNorm1d(512,affine=False), # NewAdd(1) # Affine=False NA13
            nn.LeakyReLU(),
            nn.Dropout(p=0.1), # NewAdd(2)

            nn.Linear(512,1024), # Earlier it was 40
            nn.BatchNorm1d(1024,affine=False), # NA-14
            nn.LeakyReLU(),# NA-15
            nn.Dropout(p=0.1),# NA-16

            nn.Linear(1024,2048), # Earlier it was 40
            nn.BatchNorm1d(2048,affine=False), # NA-14
            nn.ReLU(),# NA-15
            nn.Dropout(p=0.1),# NA-16

            nn.Linear(2048,4096), # Earlier it was 40
            nn.BatchNorm1d(4096,affine=False), # NA-14
            nn.ReLU(),# NA-15
            nn.Dropout(p=0.1),# NA-16

            nn.Linear(4096,2048), # Earlier it was 40
            nn.BatchNorm1d(2048,affine=False), # NA-14
            nn.ReLU(),# NA-15
            nn.Dropout(p=0.1),# NA-16

            nn.Linear(2048,1024), # Earlier it was 40
            nn.BatchNorm1d(1024,affine=False), # NA-14
            nn.ReLU(),# NA-15
            nn.Dropout(p=0.1),# NA-16

            nn.Linear(1024,512), # Earlier it was 40
            nn.BatchNorm1d(512,affine=False), # NA-14
            nn.LeakyReLU(),# NA-15
            nn.Dropout(p=0.1),# NA-16

            nn.Linear(512,256), # Earlier it was 40
            nn.BatchNorm1d(256,affine=False), # NA-14
            nn.LeakyReLU(),# NA-15
            nn.Dropout(p=0.1),# NA-16

            nn.Linear(256,40),# NA-17
        ]
        self.laysers = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.laysers(A0)
        return x

    


In [19]:
class LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=20000, shuffle=True, partition="dev-clean", csvpath=None):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample 
        
        self.X_dir = data_path + "/" + partition + "/mfcc/"
        self.Y_dir = data_path + "/" + partition +"/transcript/"
        
        self.X_names = os.listdir(self.X_dir)
        if (partition=="train-clean-100" or partition=="dev-clean"): 
            self.Y_names = os.listdir(self.Y_dir)

        # using a small part of the dataset to debug
        
        if csvpath:
            if (partition=="train-clean-100" or partition=="dev-clean"):
                subset = self.parse_csv(csvpath)
                self.X_names = [i for i in self.X_names if i in subset]
                self.Y_names = [i for i in self.Y_names if i in subset]
            else:
                self.X_names = list(pd.read_csv(csvpath).file)
                      
        if shuffle == True:
            XY_names = list(zip(self.X_names, self.Y_names))
            random.shuffle(XY_names)
            self.X_names, self.Y_names = zip(*XY_names)
            
        assert(len(self.X_names) == len(self.Y_names))
        self.length = len(self.X_names)
        self.PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
      
    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[0])
        return subset[1:]

    def __len__(self):
        return int(np.ceil(self.length / self.sample))
        
    def __getitem__(self, i):
        sample_range = range(i*self.sample, min((i+1)*self.sample, self.length))
                  
        X, Y = [], []
        for j in sample_range:
            X_path = self.X_dir + self.X_names[j]
            Y_path = self.Y_dir + self.Y_names[j]
            
            label = [self.PHONEMES.index(yy) for yy in np.load(Y_path)][1:-1]

            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
            X.append(X_data)
            Y.append(np.array(label))
        
        X, Y = np.concatenate(X), np.concatenate(Y)
        return X, Y
    
class LibriItems(torch.utils.data.Dataset):
    def __init__(self, X, Y, context = 0):
        assert(X.shape[0] == Y.shape[0])
        
        self.length  = X.shape[0]
        self.context = context
       
        if context == 0:
            self.X, self.Y = X, Y
        else:
            # TODO: self.X, self.Y = ... 
            T1 = np.array(np.zeros((context,13)))
            self.X = np.concatenate((T1,X,T1),axis=0)
            self.Y = Y
            #pass
        
                
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten('C')
            yy = self.Y[i]
        else:
            # TODO xx, yy = ...
            xx = self.X[i:(2*self.context)+i+1].flatten('C')
            yy = self.Y[i]
            #pass
        return xx,yy # return xx only for test
    



In [ ]:
class LibriSamplestest(torch.utils.data.Dataset):
    def __init__(self, data_path, shuffle=True, partition="test-clean", csvpath=None):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        #self.sample = sample 
        
        self.X_dir = data_path + "/" + partition + "/mfcc/"
                
        self.X_names = os.listdir(self.X_dir)
       
        # using a small part of the dataset to debug
        
        if csvpath:
            self.X_names = list(pd.read_csv(csvpath).file)
                      
        self.length = len(self.X_names)
        #self.PHONEMES = [
        #    'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
        #    'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
        #    'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
        #    'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
        #    'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
        #    'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
      
    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[0])
        return subset[1:]

    def __len__(self):
        return int(np.ceil(self.length / self.sample))
        
    def __getitem__(self, i):
        #sample_range = range(i*self.sample, min((i+1)*self.sample, self.length))
                  
        X = []
        for j in sample_range:
            X_path = self.X_dir + self.X_names[j]
                        
            #label = [self.PHONEMES.index(t) for t in np.load(X_path)][1:-1]

            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
            X.append(X_data)
        
        X= np.concatenate(X)
        return X

In [20]:
class LibriItemstest(torch.utils.data.Dataset):
    def __init__(self, X, context = 0):
               
        self.length  = len(X)
        self.context = context
       
        if context == 0:
            self.X = X
        else:
            # TODO: self.X, self.Y = ... 
            T1 = np.array(np.zeros((context,13)))
            self.X = np.concatenate((T1,X,T1),axis=0)
            #pass
        
                
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten('C')
        else:
            # TODO xx, yy = ...
            xx = self.X[i:(2*self.context)+i+1].flatten('C')
            #pass
        return xx # return xx only for test
    

In [21]:
class LibriItemsTestV2(torch.utils.data.Dataset):
    def __init__(self, data_path, partition, context, LIBRIPATH):
        #assert(X.shape[0] == Y.shape[0])
        df = pd.read_csv(LIBRIPATH)
        
        files = list(df.file)
        
        self.X = []
        
        for t in files:
            r = np.load(data_path + "/" + partition + "/mfcc/"+t)
            r = (r - r.mean(axis=0))/r.std(axis=0)
            self.X.append(r)
        
        self.X=np.concatenate(self.X)
        self.length  = self.X.shape[0]
        self.X = np.pad(self.X,((context,context),(0,0)),mode="constant",constant_values=(0,0))
        
        self.context = context
                    
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
            xx = self.X[i:(2*self.context)+i+1].flatten()
          
            return xx # return xx only for test

In [ ]:
!mkdir data

In [1]:
import json

TOKEN = {"username":"mangalamsahai","key":"521f66540469b3a12f7b11566d8b1c14"}

! pip install kaggle==1.5.12
! mkdir -p .kaggle
! mkdir -p /content & mkdir -p /content/.kaggle & mkdir -p /root/.kaggle

with open('/content/.kaggle/kaggle.json','w') as file:
    json.dump(TOKEN, file)

! pip install --upgrade --force-reinstall --no-deps kaggle
! ls "/content/.kaggle"
! chmod 600 /content/.kaggle/kaggle.json
! cp /content/.kaggle/kaggle.json /root/.kaggle/

! kaggle config set -n path -v /content

     |████████████████████████████████| 58 kB 5.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=ee87bf89598551cbac2daf64cf273410bee9763faffadf24da9e310bb75e1ae6
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
kaggle.json
- path is now set to: /content


In [3]:
! unzip competitions/11-785-s22-hw1p2/11-785-s22-hw1p2.zip

Streaming output truncated to the last 5000 lines.
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-020.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-021.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-022.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-023.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-024.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-025.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-026.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-027.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-028.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-029.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript/7113-086041-030.npy  
  inflating: hw1p2_student_data/train-clean-100/transcript

In [2]:
! kaggle competitions download -c 11-785-s22-hw1p2

100% 1.85G/1.86G [00:10<00:00, 168MB/s]
100% 1.86G/1.86G [00:10<00:00, 188MB/s]


In [23]:
def train(args, model, device, train_samples, optimizer, criterion, epoch,num_workers,scaler):
    model.train()
    for i in range(len(train_samples)):
        X, Y = train_samples[i]
        train_items = LibriItems(X, Y, context=args['context'])
        train_loader = torch.utils.data.DataLoader(train_items, batch_size=args['batch_size'], shuffle=True,num_workers=num_workers) # Num_Workers change-NA11
        #scaler = GradScaler() # NA 3
        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.float().to(device)
            target = target.long().to(device)

            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                  output = model(data)
            loss = criterion(output, target)
            #loss.backward()
            scaler.scale(loss).backward() # NA 4
            scaler.step(optimizer) # NA 5
            scaler.update() # NA 6
            # optimizer.step() # NA 7 # NA-25 Remove this statement
            
            if batch_idx % args['log_interval'] == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
            torch.cuda.empty_cache() # NA 22


def test(args, model, device, dev_samples,num_workers):
    model.eval()
    true_y_list = []
    pred_y_list = []
    with torch.no_grad():
        for i in range(len(dev_samples)):
            X, Y = dev_samples[i]

            test_items = LibriItems(X, Y, context=args['context'])
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False,num_workers=num_workers)

            for data, true_y in test_loader:
                data = data.float().to(device)
                true_y = true_y.long().to(device)                
                
                output = model(data)
                pred_y = torch.argmax(output, axis=1)

                pred_y_list.extend(pred_y.tolist())
                true_y_list.extend(true_y.tolist())

    train_accuracy =  accuracy_score(true_y_list, pred_y_list)
    return train_accuracy 



def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_workers = 8 if "cuda" else 0 # NA-11
    # torch.backends.cudnn.benchmark= True # NA-12
    model = Network() # .todevice()
    model.apply(init_weights) #NA-21 #NA-23 (initialize_weights)
    model =model.to(device) # NA-24
    optimizer = optim.Adam(model.parameters(), lr=args['lr'])
    criterion = torch.nn.CrossEntropyLoss()
    scaler = GradScaler() # NA 24
    # If you want to use full Dataset, please pass None to csvpath
    train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="train-clean-100", csvpath=None)
    dev_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="dev-clean")
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1) # NA 8
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=2, mode='max', verbose=True) # NA-18 # FACTOR CHANGED TO 0.5 FROM 0.1
    for epoch in range(1, args['epoch'] + 1):
        train(args, model, device, train_samples, optimizer, criterion, epoch,num_workers,scaler) # Changes from NA-11
        test_acc = test(args, model, device, dev_samples,num_workers) # Changes from NA-11
        print('Dev accuracy ', test_acc)
        torch.save(model.state_dict(),"/content/model_epoch_"+str(epoch)+".pth")
        #scheduler.step() # NA 9
        scheduler.step(test_acc) # NA-19
        # torch.cuda.empty_cache() # NA 10

        
    
    for id,(x,y) in enumerate(dev_samples):
        labels=y
        break
    print(np.bincount(labels))
    print(np.unique(labels, return_counts=True))
    

if __name__ == '__main__':
    args = {
        'batch_size': 2048,
        'context': context,
        'log_interval': 200,
        'LIBRI_PATH': '/content/hw1p2_student_data',
        'lr': 0.001,
        'epoch': 50
    }
    main(args)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train Epoch: 1 [0/25371398 (0%)]	Loss: 4.074219
Train Epoch: 1 [409600/25371398 (2%)]	Loss: 1.286133
Train Epoch: 1 [819200/25371398 (3%)]	Loss: 1.131836
Train Epoch: 1 [1228800/25371398 (5%)]	Loss: 1.029297
Train Epoch: 1 [1638400/25371398 (6%)]	Loss: 0.976074
Train Epoch: 1 [2048000/25371398 (8%)]	Loss: 0.925293
Train Epoch: 1 [2457600/25371398 (10%)]	Loss: 0.996582
Train Epoch: 1 [2867200/25371398 (11%)]	Loss: 0.880859
Train Epoch: 1 [3276800/25371398 (13%)]	Loss: 0.926270
Train Epoch: 1 [3686400/25371398 (15%)]	Loss: 0.862793
Train Epoch: 1 [4096000/25371398 (16%)]	Loss: 0.889160
Train Epoch: 1 [4505600/25371398 (18%)]	Loss: 0.836426
Train Epoch: 1 [4915200/25371398 (19%)]	Loss: 0.880371
Train Epoch: 1 [5324800/25371398 (21%)]	Loss: 0.792969
Train Epoch: 1 [5734400/25371398 (23%)]	Loss: 0.826172
Train Epoch: 1 [6144000/25371398 (24%)]	Loss: 0.791504
Train Epoch: 1 [6553600/25371398 (26%)]	Loss: 0.770508
Train Epoch: 1 [6963200/25371398 (27%)]	Loss: 0.812012
Train Epoch: 1 [7372800/

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train Epoch: 2 [0/25371398 (0%)]	Loss: 0.620605
Train Epoch: 2 [409600/25371398 (2%)]	Loss: 0.633301
Train Epoch: 2 [819200/25371398 (3%)]	Loss: 0.659668
Train Epoch: 2 [1228800/25371398 (5%)]	Loss: 0.623047
Train Epoch: 2 [1638400/25371398 (6%)]	Loss: 0.607910
Train Epoch: 2 [2048000/25371398 (8%)]	Loss: 0.618652
Train Epoch: 2 [2457600/25371398 (10%)]	Loss: 0.615723
Train Epoch: 2 [2867200/25371398 (11%)]	Loss: 0.626953
Train Epoch: 2 [3276800/25371398 (13%)]	Loss: 0.598633
Train Epoch: 2 [3686400/25371398 (15%)]	Loss: 0.608398
Train Epoch: 2 [4096000/25371398 (16%)]	Loss: 0.596680
Train Epoch: 2 [4505600/25371398 (18%)]	Loss: 0.610840
Train Epoch: 2 [4915200/25371398 (19%)]	Loss: 0.601074
Train Epoch: 2 [5324800/25371398 (21%)]	Loss: 0.575195
Train Epoch: 2 [5734400/25371398 (23%)]	Loss: 0.604004
Train Epoch: 2 [6144000/25371398 (24%)]	Loss: 0.627441
Train Epoch: 2 [6553600/25371398 (26%)]	Loss: 0.632324
Train Epoch: 2 [6963200/25371398 (27%)]	Loss: 0.595215
Train Epoch: 2 [7372800/

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


...

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train Epoch: 15 [0/25371398 (0%)]	Loss: 0.426514
Train Epoch: 15 [409600/25371398 (2%)]	Loss: 0.380371
Train Epoch: 15 [819200/25371398 (3%)]	Loss: 0.381592
Train Epoch: 15 [1228800/25371398 (5%)]	Loss: 0.392822
Train Epoch: 15 [1638400/25371398 (6%)]	Loss: 0.418701
Train Epoch: 15 [2048000/25371398 (8%)]	Loss: 0.420654
Train Epoch: 15 [2457600/25371398 (10%)]	Loss: 0.414795
Train Epoch: 15 [2867200/25371398 (11%)]	Loss: 0.355469
Train Epoch: 15 [3276800/25371398 (13%)]	Loss: 0.412598
Train Epoch: 15 [3686400/25371398 (15%)]	Loss: 0.379883
Train Epoch: 15 [4096000/25371398 (16%)]	Loss: 0.403320
Train Epoch: 15 [4505600/25371398 (18%)]	Loss: 0.403076
Train Epoch: 15 [4915200/25371398 (19%)]	Loss: 0.408203
Train Epoch: 15 [5324800/25371398 (21%)]	Loss: 0.423828
Train Epoch: 15 [5734400/25371398 (23%)]	Loss: 0.388672
Train Epoch: 15 [6144000/25371398 (24%)]	Loss: 0.410400
Train Epoch: 15 [6553600/25371398 (26%)]	Loss: 0.440430
Train Epoch: 15 [6963200/25371398 (27%)]	Loss: 0.383789
Train 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train Epoch: 16 [0/25371398 (0%)]	Loss: 0.409180
Train Epoch: 16 [409600/25371398 (2%)]	Loss: 0.396973
Train Epoch: 16 [819200/25371398 (3%)]	Loss: 0.438721
Train Epoch: 16 [1228800/25371398 (5%)]	Loss: 0.428223
Train Epoch: 16 [1638400/25371398 (6%)]	Loss: 0.386719
Train Epoch: 16 [2048000/25371398 (8%)]	Loss: 0.375977
Train Epoch: 16 [2457600/25371398 (10%)]	Loss: 0.453613
Train Epoch: 16 [2867200/25371398 (11%)]	Loss: 0.398926
Train Epoch: 16 [3276800/25371398 (13%)]	Loss: 0.420166
Train Epoch: 16 [3686400/25371398 (15%)]	Loss: 0.388184
Train Epoch: 16 [4096000/25371398 (16%)]	Loss: 0.402588
Train Epoch: 16 [4505600/25371398 (18%)]	Loss: 0.363525
Train Epoch: 16 [4915200/25371398 (19%)]	Loss: 0.433594
Train Epoch: 16 [5324800/25371398 (21%)]	Loss: 0.433105
Train Epoch: 16 [5734400/25371398 (23%)]	Loss: 0.390381
Train Epoch: 16 [6144000/25371398 (24%)]	Loss: 0.425049
Train Epoch: 16 [6553600/25371398 (26%)]	Loss: 0.416260
Train Epoch: 16 [6963200/25371398 (27%)]	Loss: 0.373291
Train 

KeyboardInterrupt: ignored

# New Section

In [ ]:
def initialize_weights(m): # NA-20
      #weight=torch.full(((2*context+1)*13,1024),1)
      #bias = torch.full((1,1),0)
      if isinstance(m,nn.Linear):
         nn.init.kaiming_normal_(m.weight,mode='fan_out',nonlinearity='relu')
         #nn.init.constant_(weight,1)
         #nn.init.constant_(bias,0)
      #elif isinstance(m,nn.BatchNorm1d):
        #nn.init.constant_(weight,1)
        #nn.init.constant_(bias,0)

In [22]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

#net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2))
model.apply(init_weights)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


Network(
  (laysers): Sequential(
    (0): Linear(in_features=845, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.01)
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=1024, out_features=2048, bias=True)
    (9): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.01)
    (11): Dropout(p=0.1, inplace=False)
    (12): Linear(in_features=2048, out_features=4096, bias=True)
    (13): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (14): LeakyReLU(negative_slope=0.01)
    (15): Dropout(p=0.1, inplace=False)
    (16): Linear(in_feat

In [25]:
device = torch.device("cuda")
model = Network().cuda() # NA 18 cuda()
model.load_state_dict(torch.load("/content/model_epoch_16.pth"))

<All keys matched successfully>

In [26]:
def testpred(args, model, device):
    model.eval()
    pred_y_list = []
    with torch.no_grad():
            test_items = LibriItemsTestV2("/content/hw1p2_student_data","test-clean",context, "/content/test_order.csv")   
            
            #print(len(test_items[0]))
            #print(len(test_items[1]))
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)
            
            for data in test_loader:
                data = data.float().to(device)
                
                output = model(data)
                pred_y = torch.argmax(output, axis=1)

                pred_y_list.extend(pred_y.tolist())
        
    return pred_y_list 

In [27]:
output1 = testpred(args,model,device) 

In [ ]:
len(output1)

In [28]:
df=pd.DataFrame()

In [29]:
df['id'] = range(len(output1))

In [ ]:
len(df)

In [30]:
df['label'] = output1
len(df['label'])

1943253

In [ ]:
len(df)

In [31]:
df.to_csv("results.csv",index=False)

In [ ]:
df.to_pickle('results.zip')

In [32]:
! kaggle competitions submit -c 11-785-s22-hw1p2 -f results.csv -m "New Submission"

100% 18.6M/18.6M [00:02<00:00, 6.68MB/s]
Successfully submitted to Frame-Level Speech Recognition

In [ ]:
dev_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="dev-clean")

In [ ]:
for id,(x,y) in enumerate(dev_samples):
    labels=y
    break
print(np.bincount(labels))
id,vals= np.unique(labels, return_counts=True)

In [ ]:
df1=pd.DataFrame()

In [ ]:
df1['ids']=id
df1['vals']=vals

In [ ]:
print(df1)

In [ ]:
df1.sort_values(by='vals',ascending=False)

,ids,vals
0,0,396367
29,29,111876
3,3,88799
23,23,85424
31,31,84166
18,18,81199
21,21,73571
17,17,63752
38,38,59360
28,28,59326


In [ ]:
PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']

In [ ]:
PHONEMES[39]

'ZH'

In [ ]:
def train(args, model, device, train_samples, optimizer, criterion, epoch):
    model.train()
    for i in range(len(train_samples)):
        X, Y = train_samples[i]
        train_items = LibriItems(X, Y, context=args['context'])
        train_loader = torch.utils.data.DataLoader(train_items, batch_size=args['batch_size'], shuffle=True)

        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.float().to(device)
            target = target.long().to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % args['log_interval'] == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, dev_samples):
    model.eval()
    true_y_list = []
    pred_y_list = []
    with torch.no_grad():
        for i in range(len(dev_samples)):
            X, Y = dev_samples[i]

            test_items = LibriItems(X, Y, context=args['context'])
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)

            for data, true_y in test_loader:
                data = data.float().to(device)
                true_y = true_y.long().to(device)                
                
                output = model(data)
                pred_y = torch.argmax(output, axis=1)

                pred_y_list.extend(pred_y.tolist())
                true_y_list.extend(true_y.tolist())

    train_accuracy =  accuracy_score(true_y_list, pred_y_list)
    return train_accuracy 

def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = Network().to(device)
    optimizer = optim.Adam(model.parameters(), lr=args['lr'])
    criterion = torch.nn.CrossEntropyLoss()
    # If you want to use full Dataset, please pass None to csvpath
    train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="train-clean-100", csvpath='/content/train_filenames_subset_8192_v2.csv')
    dev_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="dev-clean")
       
    for epoch in range(1, args['epoch'] + 1):
        train(args, model, device, train_samples, optimizer, criterion, epoch)
        test_acc = test(args, model, device, dev_samples)
        print('Dev accuracy ', test_acc)
    
    
    for id,(x,y) in enumerate(dev_samples):
        labels=y
        break
    print(np.bincount(labels))
    print(np.unique(labels, return_counts=True))
    

if __name__ == '__main__':
    args = {
        'batch_size': 2048,
        'context': context,
        'log_interval': 200,
        'LIBRI_PATH': '/content/hw1p2_student_data',
        'lr': 0.001,
        'epoch': 3
    }
    main(args)

Train Epoch: 1 [0/10391263 (0%)]	Loss: 3.696327
Train Epoch: 1 [409600/10391263 (4%)]	Loss: 2.021652
Train Epoch: 1 [819200/10391263 (8%)]	Loss: 1.845060
Train Epoch: 1 [1228800/10391263 (12%)]	Loss: 1.737632
Train Epoch: 1 [1638400/10391263 (16%)]	Loss: 1.827224
Train Epoch: 1 [2048000/10391263 (20%)]	Loss: 1.783051
Train Epoch: 1 [2457600/10391263 (24%)]	Loss: 1.711157
Train Epoch: 1 [2867200/10391263 (28%)]	Loss: 1.811529
Train Epoch: 1 [3276800/10391263 (32%)]	Loss: 1.729257
Train Epoch: 1 [3686400/10391263 (35%)]	Loss: 1.709938
Train Epoch: 1 [4096000/10391263 (39%)]	Loss: 1.763428
Train Epoch: 1 [4505600/10391263 (43%)]	Loss: 1.779171
Train Epoch: 1 [4915200/10391263 (47%)]	Loss: 1.760154
Train Epoch: 1 [5324800/10391263 (51%)]	Loss: 1.663584
Train Epoch: 1 [5734400/10391263 (55%)]	Loss: 1.729900
Train Epoch: 1 [6144000/10391263 (59%)]	Loss: 1.742244
Train Epoch: 1 [6553600/10391263 (63%)]	Loss: 1.684746
Train Epoch: 1 [6963200/10391263 (67%)]	Loss: 1.774129
Train Epoch: 1 [73728